model_name = mistralai/Mistral-7B-Instruct-v0.3

Install Dependencies

In [1]:
!pip install accelerate peft bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [2]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


HuggingFace Login

In [3]:
from huggingface_hub import login
login()

In [4]:
import torch
print("CUDA:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

CUDA: True
GPU: NVIDIA A100-SXM4-40GB


In [5]:
import numpy as np
import pandas as pd

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

NumPy version: 2.0.2
Pandas version: 2.2.2


In [6]:
pip show transformers accelerate peft

Name: transformers
Version: 4.52.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers
---
Name: accelerate
Version: 1.7.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft
---
Name: peft
Version: 0.15.2
Summary: Parameter-Efficient Fine-Tun

Import Libraries

In [7]:
# Import libraries
import bitsandbytes as bnb
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed,  BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments, LlamaTokenizer, EarlyStoppingCallback
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
from datasets import load_dataset
import random
import pandas as pd

seed = 42
set_seed(seed)

Quantization

In [8]:
def create_bnb_config():
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

Model Loader and Tokenizer

In [9]:
def load_model(model_name, bnb_config):
    print("N GPUS:", torch.cuda.device_count())

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True  # Qwen için mutlaka True
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_fast=True,
        trust_remote_code=True  # Qwen için mutlaka True
    )

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

Prompt Format and Dataset

Llama2 prompt format: [INST] {instruction}\n\n{sentence} [/INST] {answer}
Llama3 prompt format: <|begin_of_text|><|user|>
{instruction_and_input}
<|assistant|>
{expected_output}


Adding Qwen Prompt format (may need to double check)

In [10]:
def create_prompt_formats(sample):
    instruction = "What is the key word that represents the interaction between the proteins which are tagged with [Protein1] and [Protein2] in the given sentence?"
    user_prompt = f"{instruction}\n\n{sample['Sentence']}"
    answer = sample['Keywords']

    sample["text"] = tokenizer.apply_chat_template(
        [{"role": "user", "content": user_prompt}, {"role": "assistant", "content": answer}],
        tokenize=False,
        add_generation_prompt=False
    )
    return sample

Dataset processing

In [18]:
def preprocess_batch(batch, tokenizer, max_length):
    return tokenizer(batch["text"], max_length=max_length, truncation=True)

from functools import partial

def preprocess_dataset(tokenizer, max_length, seed, dataset):
    dataset = dataset.map(create_prompt_formats, batched=False)  # ✅ Burada batched=False
    print("Sample prompt:\n", dataset[0]["text"])
    f = partial(preprocess_batch, tokenizer=tokenizer, max_length=max_length)
    dataset = dataset.map(f, batched=True)
    dataset = dataset.filter(lambda x: len(x["input_ids"]) < max_length)
    return dataset.shuffle(seed=seed)

LoRA Settings

In [27]:
def create_peft_config(target_modules):
    return LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=["w1", "w2", "w3", "out_proj", "lm_head"], # target modules for qwen are different
        lora_dropout=0.0,
        bias="none",
        task_type="CAUSAL_LM",
    )

def find_all_linear_names(model):
    cls = torch.nn.Linear
    return list({name.split(".")[-1] for name, mod in model.named_modules() if isinstance(mod, cls)})

Optimized Train Function for Qwen and A100 Support

In [24]:
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(name, type(module))

model.layers.0.self_attn.q_proj.base_layer.base_layer <class 'bitsandbytes.nn.modules.Linear4bit'>
model.layers.0.self_attn.q_proj.base_layer.lora_A.default <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.q_proj.base_layer.lora_B.default <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.q_proj.lora_A.default <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.q_proj.lora_B.default <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.k_proj.base_layer <class 'bitsandbytes.nn.modules.Linear4bit'>
model.layers.0.self_attn.k_proj.lora_A.default <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.k_proj.lora_B.default <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.v_proj.base_layer <class 'bitsandbytes.nn.modules.Linear4bit'>
model.layers.0.self_attn.v_proj.lora_A.default <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.v_proj.lora_B.default <class 'torch.nn.modules.linear.Linear'>

In [28]:
def train(model, tokenizer, train_dataset, val_dataset, output_dir):
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)

    # Qwen modellerinde genellikle Linear modüllerin isimleri farklı olabilir. find_all_linear_names() ile kontrol et.
    target_modules = ["w1", "w2", "w3", "out_proj", "lm_head"] # Target modules for qwen
    print("Target modules for LoRA:", target_modules)

    peft_config = create_peft_config(target_modules)
    model = get_peft_model(model, peft_config)

    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable params: {trainable:,} / {total:,} ({100 * trainable / total:.2f}%)")

    trainer = Trainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        args=TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=4,
            num_train_epochs=3,
            gradient_accumulation_steps=4,
            learning_rate=2e-4,
            bf16=True,
            fp16=False,
            logging_steps=1,
            eval_strategy="steps",
            save_strategy="steps",
            eval_steps=5,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            report_to=None
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )

    trainer.train()
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)


Load Dataset

In [15]:
# Dataset
data_files = {
    "train": "train.jsonl",
    "test": "test.jsonl"
}
from datasets import load_dataset

dataset = load_dataset("bengisucam/LLL_INO-tagged", data_files=data_files, cache_dir=None, download_mode='force_redownload')

# Access the splits directly from the loaded dataset
train_dataset = dataset["train"]
val_dataset = dataset["test"]

print(train_dataset[:2])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.jsonl:   0%|          | 0.00/68.5k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/12.0k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{'Unnamed: 0': [0, 1], 'Gene1': ['KinC', 'KinD'], 'Gene2': ['Spo0A~P', 'Spo0A~P'], 'Keywords': ['responsible, production', 'responsible, production'], 'Direct dependency relations': ['nmod(responsible, production)', 'nmod(responsible, production)'], 'INO Type': ['regulation of translation', 'regulation of translation'], 'INO ID': ['INO_0000034', 'INO_0000034'], 'Num. Keywords': ['2', '2'], 'Sentence': ['In vivo studies of the activity of four of the kinases, KinA, [PROTEIN1] KinC [/PROTEIN1], KinD (ykvD) and KinE (ykrQ), using abrB transcription as an indicator of [PROTEIN2] Spo0A~P [/PROTEIN2] level, revealed that KinC and KinD were responsible for Spo0A~P production during the exponential phase of growth in the absence of KinA and KinB.', 'In vivo studies of the activity of four of the kinases, KinA, KinC, [PROTEIN1] KinD [/PROTEIN1] (ykvD) and KinE (ykrQ), using abrB transcription as an indicator of [PROTEIN2] Spo0A~P [/PROTEIN2] level, revealed that KinC and KinD were responsible f

Load Model

In [16]:
# Qwen 2.5 7B 1M Token Instruct Model
model_name = "Qwen/Qwen2.5-7B-Instruct-1M"
bnb_config = create_bnb_config()
model, tokenizer = load_model(model_name, bnb_config)

N GPUS: 1


config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Preprocess the Dataset

In [19]:
# Preprocessing
max_length = 1024
train_dataset_processed = preprocess_dataset(tokenizer, max_length, seed, train_dataset)
val_dataset_processed = preprocess_dataset(tokenizer, max_length, seed, val_dataset)

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

Sample prompt:
 <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the key word that represents the interaction between the proteins which are tagged with [Protein1] and [Protein2] in the given sentence?

In vivo studies of the activity of four of the kinases, KinA, [PROTEIN1] KinC [/PROTEIN1], KinD (ykvD) and KinE (ykrQ), using abrB transcription as an indicator of [PROTEIN2] Spo0A~P [/PROTEIN2] level, revealed that KinC and KinD were responsible for Spo0A~P production during the exponential phase of growth in the absence of KinA and KinB.<|im_end|>
<|im_start|>assistant
responsible, production<|im_end|>



Map:   0%|          | 0/139 [00:00<?, ? examples/s]

Filter:   0%|          | 0/139 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Sample prompt:
 <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the key word that represents the interaction between the proteins which are tagged with [Protein1] and [Protein2] in the given sentence?

In addition to the typical [PROTEIN1] sigmaB [/PROTEIN1]-dependent, stress- and starvation-inducible pattern, [PROTEIN2] yvyD [/PROTEIN2] is also induced in response to amino acid depletion.<|im_end|>
<|im_start|>assistant
dependent, inducible<|im_end|>



Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25 [00:00<?, ? examples/s]

Train

In [20]:
from transformers import TrainingArguments
print(TrainingArguments.__init__.__code__.co_varnames)

('self', 'output_dir', 'overwrite_output_dir', 'do_train', 'do_eval', 'do_predict', 'eval_strategy', 'prediction_loss_only', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'per_gpu_train_batch_size', 'per_gpu_eval_batch_size', 'gradient_accumulation_steps', 'eval_accumulation_steps', 'eval_delay', 'torch_empty_cache_steps', 'learning_rate', 'weight_decay', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'max_grad_norm', 'num_train_epochs', 'max_steps', 'lr_scheduler_type', 'lr_scheduler_kwargs', 'warmup_ratio', 'warmup_steps', 'log_level', 'log_level_replica', 'log_on_each_node', 'logging_dir', 'logging_strategy', 'logging_first_step', 'logging_steps', 'logging_nan_inf_filter', 'save_strategy', 'save_steps', 'save_total_limit', 'save_safetensors', 'save_on_each_node', 'save_only_model', 'restore_callback_states_from_checkpoint', 'no_cuda', 'use_cpu', 'use_mps_device', 'seed', 'data_seed', 'jit_mode_eval', 'use_ipex', 'bf16', 'fp16', 'fp16_opt_level', 'half_precision_backend',

In [29]:
# Train
output_dir = "results/qwen2.5_colab/"
train(model, tokenizer, train_dataset_processed, val_dataset_processed, output_dir)

Target modules for LoRA: ['w1', 'w2', 'w3', 'out_proj', 'lm_head']
Trainable params: 21,487,616 / 4,374,459,904 (0.49%)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: busra-oguzoglu (busra-ai-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
5,1.734700,1.441218
10,1.003500,0.932895
15,0.792400,0.801653
20,0.569300,0.732333
25,0.627500,0.695054


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TEST

In [30]:
import torch
from transformers import  LlamaTokenizer, set_seed
from peft import  AutoPeftModelForCausalLM
from datasets import load_dataset
from datetime import datetime as dt
import logging

In [44]:
# Reproducibility
seed = 42
set_seed(seed)

# Prompt oluşturucu
def create_prompt_formats_for_test(sample):
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction: What is the key word that represents the interaction between the proteins which are tagged with [Protein1] and [Protein2] in the given sentence?"
    INPUT_KEY = "### Input:"
    RESPONSE_KEY = "### Response:"
    sample["text"] = "\n\n".join([
        INTRO_BLURB,
        INSTRUCTION_KEY,
        f"{INPUT_KEY}\n{sample['Sentence']}",
        f"{RESPONSE_KEY}\n"
    ])
    return sample

In [47]:
# Log ayarları
logging.basicConfig(filename="finetune_results/qwen2.5-light-infer.log", level=logging.INFO)
logging.info(f"({dt.now().strftime('%d/%m/%Y %H:%M:%S')}) | START")

# Cihaz tanımı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model yolu
tmp_model_path = "results/qwen2.5_colab/"
print("Loading Qwen2.5 model checkpoint with minimal memory...")

# Model yükle
model = AutoPeftModelForCausalLM.from_pretrained(
    tmp_model_path,
    low_cpu_mem_usage=True
)
model.eval()  # inference modu
model.to(device)

# Tokenizer yükle
tokenizer = AutoTokenizer.from_pretrained(
    tmp_model_path,
    use_fast=True,
    trust_remote_code=False
)

# Dataset
test_dataset = load_dataset("bengisucam/LLL_INO-tagged", split="test")
test_dataset = test_dataset.map(create_prompt_formats_for_test)
print("Dataset size:", len(test_dataset))

# Inference
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    text = sample["text"]
    sentence_id = sample["Unnamed: 0"]

    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

    # Çıktıyı çöz
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.split("[/INST]")[-1].strip() if "[/INST]" in decoded else decoded

    print(f"EXAMPLE {i+1} | Sentence ID: {sentence_id}")
    print(response)
    print("###############################")

    logging.info("Sentence Id: %s | Response: %s", sentence_id, response)

    # GPU belleğini boşalt
    del inputs, outputs
    torch.cuda.empty_cache()

Loading Qwen2.5 model checkpoint with minimal memory...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Dataset size: 25


...

EXAMPLE 22 | Sentence ID: 18
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: What is the key word that represents the interaction between the proteins which are tagged with [Protein1] and [Protein2] in the given sentence?

### Input:
Our results indicate that the level of [PROTEIN2] KatX [/PROTEIN2] level in outgrowing spores depends mainly on [PROTEIN1] EsigmaF [/PROTEIN1], because sigB mutants show normal KatX activity in dormant and outgrowing spores.

### Response:
The key word that represents the interaction between the proteins [Protein1] (EsigmaF) and [Protein2] (KatX) in the given sentence is "depends." This indicates that the level of KatX is influenced by Es
###############################


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


EXAMPLE 23 | Sentence ID: 12
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: What is the key word that represents the interaction between the proteins which are tagged with [Protein1] and [Protein2] in the given sentence?

### Input:
In the shaA mutant, [PROTEIN1] sigma(H) [/PROTEIN1]-dependent expression of [PROTEIN2] spo0A [/PROTEIN2] and spoVG at an early stage of sporulation was sensitive to external NaCl.

### Response:
The key word that represents the interaction between the proteins [Protein1] (sigma(H)) and [Protein2] (spo0A and spoVG) in the given sentence is "dependent." This indicates that the expression of spo0A
###############################


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


EXAMPLE 24 | Sentence ID: 9
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: What is the key word that represents the interaction between the proteins which are tagged with [Protein1] and [Protein2] in the given sentence?

### Input:
Analysis of the expression of a translational ywhE-lacZ fusion showed that [PROTEIN2] ywhE [PROTEIN2] expression is sporulation-specific, and is controlled predominantly by the forespore-specific sigma factor sigma(F), and to a lesser extent by [PROTEIN1] sigma(G) [/PROTEIN1].

### Response:
The key word that represents the interaction between the proteins tagged with [Protein1] and [Protein2] in the given sentence is "controlled."
###############################
EXAMPLE 25 | Sentence ID: 31
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: What is the key word that represents the interaction between the prote

In [46]:
import gc
torch.cuda.empty_cache()
gc.collect()

12310

Get environment variables

In [48]:
!pip freeze > requirements-qwen2.5_instructv3.txt